## Overstory Interview Assignment - part 2
This is a quick notebook to guide you in testing the interview assignment results. You can also find this info in the README.d

We'll be using [k3s](https://k3s.io/) below through [smol-k8s-lab](https://github.com/small-hack/smol-k8s-lab), but you could use any k8s distro that is GPU enabled.

In [ ]:
# first install a k8s distro. I use smol-k8s-lab to get rolling quickly
!pip3.11 install smol-k8s-lab

In [ ]:
# smol-k8s-lab requires a config file. we will set that up after we create the directory
!mkdir -p ~/.config/smol-k8s-lab

In [ ]:
# set this to a free IP on your network (don't forget the CIDR notation!) and then you can use it in your local DNS
# you can setup an A record for your domain in your pihole 'local DNS' if you're using that, or ping me, and I can help you with your local router!
!echo -e "metallb_address_pool:\n  - 192.168.42.42/32" > ~/.config/smol-k8s-lab/config.yaml

# this is used for SSL with lets encrypt - replace with your email
!echo "email: name@email.com" >> ~/.config/smol-k8s-lab/config.yaml

# this is the log level, which I set to debug so you can see everything going on
!echo -e "log:\n  level: debug" >> ~/.config/smol-k8s-lab/config.yaml

#### Install a k3s cluster using smol-k8s-lab
I can't put this as a notebook cell, because it will prompt for sudo in the middle of the script if you don't have passwordless sudo enabled. I can catch it, but it is also a long running cell with a few sleeps to catch k8s wait issues, so you will need to run the following in a terminal:

```bash
smol-k8s-lab k3s
```

After that, please wait for k3s to finish setting everything up, generally around up to 3 minutes (on slower machines).

In [ ]:
# apply the manifests for this assignment
!kubectl apply -f k8s_manifests/

In [ ]:
# this is optional, as I already have the tif locally, and you probably do too :)
!wget "https://20tree-public.s3-eu-west-1.amazonaws.com/candidates/cloudmask/Sentinel2L2A_sen2cor_18TUR_20180812_clouds=5.3%_area=99%.tif

In [ ]:
# use the local script made from the original assignment notebook to create a crop
!./crop_satellite_img.py -s Sentinel2L2A_sen2cor_18TUR_20180812_clouds\=5.3%_area\=99%.tif

If you didn't have time or don't know how to set up your DNS,
we can just use port forwarding to get you up and running :)
You cannot run this in a python notebook because it runs in the foreground and background processes are not supported in all notebook renderers, causing a thread lock, so you'll need to open this in a seperate terminal to run the following:

```bash
kubectl port-forward deployment/infer-sat-image-flask-app 5000:8080
```

In [ ]:
# this tests the endpoint to make sure it can process our 512x512 image
!curl -F "file=@cropped_img.tif" http://127.0.0.1:5000/infer_image/0 --output test_response.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6154k  100 1031k  100 5123k  4336k  21.0M --:--:-- --:--:-- --:--:-- 25.3M


In [ ]:
# we need numpy to test the pickle (a numpy array) that we just downloaded
import numpy as np

In [ ]:
# this is the test to ensure that the pickle works!
np.load('test_response.pkl', allow_pickle=True)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)